# Chapter 15: Context Managers and `else` Blocks

This chapter explores control flow features which are not so common in other languages, namely

* The `with` statement and context managers
* The `else` clause in `for`, `while`, and `try` statements

## Do This, Then That: `else` Blocks Beyond `if`

Here are the rules

**`for/else`:**<br>
The `else` block will run only if and when the `for` loop runs to completion (i.e. **not** if the `for` is aborted with a `break`).

**`while/else`:**<br>
The `else` block will run only if and when the `while` loop exits because the condition became *falsy* (i.e. not when the `while` is aborted with a `break`).

**`try/else`:**<br>
The `else` block will only run if no exception is raised in the `try` block.

**Note:** In all cases, the `else` clause is skipped if an exception or a `return`, `break`, or `continue` statement causes control to jump out of the main block of the compound statement.

### Examples

The use of `else` blocks in `for` loops generally follows this pattern:

```python
for item in my_list:
    if item.flavour == "banana":
        break
else:
    raise ValueError("No banana flavour found")
```

In `try/except` blocks, the `try` block should only have statements that may generate the expected exceptions. Instead of

```python
try:
    dangerous_call()
    after_call()
except OSError:
    log("OSError..." )
```

use the `else` block for clarity and correctness:

```python
try:
    dangerous_call()
except OSError:
    log("OSError..." )
else:
    after_call()
```

In Python, `try/except` is commonly used for control flow, and not just for error handling. See e.g. the [offical Python glossary](https://docs.python.org/3/glossary.html#term-eafp):

> **EAFP:**<br>Easier to ask for forgiveness than permission. This common Python coding style assumes the existence of valid keys or attributes and catches exceptions if the assumption proves false. This clean and fast style is characterized by the presence of many `try` and `except` statements. The technique contrasts with the [LBYL](https://docs.python.org/3/glossary.html#term-lbyl) style common to many other languages such as C.

## Context Managers and `with` Blocks

The `with` statement sets up a temporary context and reliably tears it down, preventing errors and reducing boilerplate code.

The **context manager protocol** exists to control a `with` statement, just like the iterator protocol exists to control a `for` statement.

The `with` statement was designed to simplify the `try/finally` pattern, which guarantees that some operation, usually releasing resources or restoring some changed state, is **always** performed after a block of code.

The context manager protocol consists of the `__enter__` and `__exit__` methods.

At the start of the `with`, `__enter__` is invoked on the context manager object.

The role of the `finally` clause is played by a call to `__exit__` on the context manager object at the end of the `with` block.

In [1]:
# Example 15-1. Demonstration of a file object as a context manager
with open("../examples/mirror.py") as fp:
    src = fp.read(60)

The **context manager object** is the result of evaluating the expression after `with`:

In [2]:
len(src)

60

The value bound to the target variable in the `as` clause is the result of calling `__enter__` on the context manager object.

In the above example, `open()` returns an instance of `TextIOWrapper`, and its `__enter__` method returns `self`. But the `__enter__` method may return some other object instead of the context manager.

In [3]:
fp

<_io.TextIOWrapper name='../examples/mirror.py' mode='r' encoding='UTF-8'>

The target variable `fp` is still available, and its attributes can be read. We can, however, **not** perform I/O with `fp`, because `TextWrapperIO.__exit__` has been called at the end of the `with` block, closing the file: 

In [4]:
fp.read(60)

ValueError: I/O operation on closed file.

When the control flow exits the `with` block in any way, the **`__exit__` method is invoked on the context manager object, not on whatever is returned by `__enter__`**.

The `as` clause is optional, except in the case of `open` which needs a reference to the file. Some context managers return `None` because they have no useful object to return to the user.

**The following example is meant to show the distinction between the context manager and the object returned by its `__enter__` method:**

In [5]:
# Example 15-2: Test driving the LookingGlass context manager class

from examples.mirror import LookingGlass
import inspect

SRC = inspect.getsource(LookingGlass)
print(SRC)

with LookingGlass() as what:
    print("Alice, Kitty and Snowdrop")
    print(what)
    
print(what)
print("Back to normal.")

class LookingGlass:
    def __enter__(self):
        import sys

        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return "JABBERWOCKY"

    def reverse_write(self, text):
        self.original_write(text[::-1])

    def __exit__(self, exc_type, exc_value, traceback):
        import sys

        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print("Please DO NOT divide by zero!")
            return True

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ
JABBERWOCKY
Back to normal.


## The `contextlib` Utilities

The [`contextlib`](https://docs.python.org/3/library/contextlib.html) module provides utilities for common tasks involving the `with` statement.

**One of the most widely used utilites is the `@contextmanager` decorator.**

It reduces the boilerplate of creating context managers. Instead of writing a whole class with `__enter__`/`__exit__` methods, simply write a generator with a single `yield` that should produce whatever you want the `__enter__` method to return.

In [6]:
# Example 15-5. a context manager implemented with a generator

from contextlib import contextmanager

@contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    
    def reverse_write(text):
        return original_write(text[::-1])
    
    sys.stdout.write = reverse_write
    msg = ""
    
    try:
        yield "JABBERWOCKY"
    except ZeroDivisionError:
        msg = "Please DO NOT divide by zero!"
    finally:
        sys.stdout.write = original_write
        if msg:
            print(msg)
    

with looking_glass() as what:
    print("Alice, Kitty and Snowdrop")
    print(what)

print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ
JABBERWOCKY


Essentially, everything up to and including the returned value from the `yield` statement is wrapped in and `__enter__` method, and everything after `yield` is wrapped in an `__exit__` method. This is greatly simplified, refer to **pp. 471-474** for details.

### Comment on material

I find this part of the book a bit hard to grasp, and the overly-engineered examples don't really have a pedagogical function. They are just confusing.

Ramalho suggests studying the [following example](https://www.zopatista.com/python/2013/11/26/inplace-file-rewriting/) of a real-life example of `@contextmanager`.

Other useful references include Raymond Hettinger's answer to ["Is it a good practice to use try-except-else in Python?"](https://stackoverflow.com/questions/16138232/is-it-a-good-practice-to-use-try-except-else-in-python) on Stack Overflow.